Abyste mohli spustit následující notebooky, pokud jste to ještě neudělali, musíte nastavit openai klíč v souboru .env jako `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Dále načteme index vektorových reprezentací (Embedding Index) do datového rámce Pandas. Index vektorových reprezentací je uložen v souboru JSON s názvem `embedding_index_3m.json`. Index vektorových reprezentací obsahuje vektory pro každý z přepisů z YouTube až do konce října 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Dále vytvoříme funkci s názvem `get_videos`, která vyhledá v Embedding Indexu podle zadaného dotazu. Funkce vrátí 5 videí, která jsou dotazu nejpodobnější. Funkce funguje následovně:

1. Nejprve se vytvoří kopie Embedding Indexu.
2. Poté se pro dotaz vypočítá embedding pomocí OpenAI Embedding API.
3. Následně se v Embedding Indexu vytvoří nový sloupec s názvem `similarity`. Tento sloupec obsahuje kosinovou podobnost mezi embeddingem dotazu a embeddingem každého video segmentu.
4. Dále se Embedding Index filtruje podle sloupce `similarity`. Zůstanou pouze videa, která mají kosinovou podobnost větší nebo rovnou 0,75.
5. Nakonec se Embedding Index seřadí podle sloupce `similarity` a vrátí se 5 nejlepších videí.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Tato funkce je velmi jednoduchá, pouze vypíše výsledky vyhledávacího dotazu.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Nejprve se Embedding Index načte do Pandas Dataframe.
2. Poté je uživatel vyzván k zadání dotazu.
3. Následně je zavolána funkce `get_videos`, která vyhledá dotaz v Embedding Indexu.
4. Nakonec je zavolána funkce `display_results`, která zobrazí výsledky uživateli.
5. Uživatel je pak znovu vyzván k zadání dalšího dotazu. Tento proces pokračuje, dokud uživatel nezadá `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.cs.png)

Budete vyzváni k zadání dotazu. Zadejte dotaz a stiskněte enter. Aplikace vrátí seznam videí, která souvisejí s vaším dotazem. Aplikace také zobrazí odkaz na místo ve videu, kde se nachází odpověď na vaši otázku.

Zde je několik dotazů, které si můžete vyzkoušet:

- Co je Azure Machine Learning?
- Jak fungují konvoluční neuronové sítě?
- Co je to neuronová síť?
- Můžu používat Jupyter Notebooks s Azure Machine Learning?
- Co je ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Prohlášení**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Za autoritativní zdroj by měl být považován původní dokument v jeho rodném jazyce. Pro kritické informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vzniklé v důsledku použití tohoto překladu.
